# main

In [1]:
import json
import requests
import popelines
import os
from main import fix_values, populating_vals, flatten, flatten_dupe_vals

In [2]:
def fix_values(value, key, reset_key):
    if key == reset_key:
        new_list = []
        for x in value:
            value[x][f'{reset_key}_id'] = x
            new_list.append(value[x])
        return new_list
    else:
        return value

In [3]:
# read endpoint, returns a json file of the HTTP request
def read_endpoint(endpoint, headers_set, params_set=None):
    try:
        response = requests.get(endpoint, headers=headers_set, params=params_set)
        response_text = json.loads(response.text)
        response.raise_for_status()

    except requests.exceptions.HTTPError as err:
        print(err)

    return response_text

In [4]:
# generate all experiments given a project id
def generate_experiments(project_id, experiment_endpoint, experiment_headers, experiment_params):
    # get all experiments from one project
    j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=experiment_headers, params_set=experiment_params)

    # loop for all experiments in this project
    experiment_id_list = []
    upload_exp_list = []
    for exp in j_exp:
        exp['project_id'] = project_id
        upload_exp_list.append(flatten(exp, {}, ''))
        experiment_id_list.append(exp['id'])
        
    return upload_exp_list, experiment_id_list

In [5]:
# generate all projects within account
def generate_projects(project_endpoint, project_headers):
    # get all projects
    j_proj = read_endpoint(endpoint=project_endpoint, headers_set=project_headers)

    # store a list of project metadata
    return j_proj

# load all projects, get project_id's

In [8]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
    'Content-Type': 'application/json',
}

In [ ]:
project_endpoint = 'https://api.optimizely.com/v2/projects'

In [ ]:
all_projects = generate_projects(project_endpoint, headers)

In [ ]:
# pope.write_to_json(file_name='../uploads/projects.json', jayson=all_projects, mode='w')

In [ ]:
# pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
# print("Successfully uploaded all projects.")

In [ ]:
project_id_list = []
for project in all_projects:
    project_id_list.append((project['id']))

In [ ]:
project_id_list


In [ ]:
project_id_name = []
for project in all_projects:
    project_id_name.append((project['id'], project['name']))

In [ ]:
project_id_name

# load experiments

In [ ]:
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'


In [ ]:
experiment_id_list = []

for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
    params = (
        ('project_id', project_id),
        ('per_page', 100),
    ) 

    exp_list, exp_id_list = generate_experiments(project_id, experiment_endpoint, headers, params)
    experiment_id_list.extend(exp_id_list)
    
    # upload experiments
    pope.write_to_json(file_name='../uploads/experiments.json', jayson=exp_list, mode='w')
    pope.write_to_bq(table_name='experiments', file_name='../uploads/experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
    print(f"Successfully uploaded experiments for project {project_id}")

# results timeseries

In [ ]:
experiment_id_list

In [6]:
result_endpoint = f'https://api.optimizely.com/v2/experiments/11479333433/timeseries'

In [9]:
response_ts = requests.get(result_endpoint, headers=headers)

# if '' then the experiment has not started yet
if response_ts.text == '':
    j_ts = {'experiment_id': experiment_id}
elif 'bad' in response_ts.text:
    j_ts = {'experiment_id': experiment_id}
else:
    j_ts = json.loads(response_ts.text)

new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')


In [10]:
from datetime import datetime

In [ ]:
flattened_metrics = []
for metric in new_j_ts['metrics']:
    for ts in metric['results']:
        flattened_timeseries = []
        for element in ts['timeseries']:
            element['upload_ts'] = str(datetime.now())
            flattened_timeseries.append(flatten(element, {}, ''))

        # Replace old 'timeseries' with new 'flattened_timeseries'
        updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
        flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')

    # Replace old 'metrics' with new 'flattened_results'
    update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
    flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))
    

In [ ]:
flattened_metrics

In [ ]:
response_ts = requests.get(result_endpoint, headers=headers)

In [ ]:
'bad' in response_ts.text

In [21]:
new_j_ts

{'confidence_threshold': 0.9,
 'end_time': '2018-09-24T23:15:00.000000Z',
 'experiment_id': 11479333433,
 'metrics': [{'aggregator': 'unique',
   'event_id': 11479433525,
   'name': 'Visit Page: Live Demo Thank you',
   'scope': 'visitor',
   'winning_direction': 'increasing'}],
 'start_time': '2018-09-24T18:30:44.000000Z',
 'stats_config': {'confidence_level': 0.9,
  'difference_type': 'relative',
  'epoch_enabled': False}}

In [19]:
flattened_metrics = []
for metric in new_j_ts['metrics']:
    if 'results' in metric.keys():
        for ts in metric['results']:
            flattened_timeseries = []
            for element in ts['timeseries']:
                element['upload_ts'] = str(datetime.now())
                flattened_timeseries.append(flatten(element, {}, ''))

            # Replace old 'timeseries' with new 'flattened_timeseries'
            updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
            flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')

        # Replace old 'metrics' with new 'flattened_results'
        update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
        flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))

    else:
        flattened_metrics = flatten(new_j_ts, {}, '')

In [20]:
flattened_metrics

{'confidence_threshold': 0.9,
 'end_time': '2018-09-24T23:15:00.000000Z',
 'experiment_id': 11479333433,
 'metrics_aggregator': 'unique',
 'metrics_event_id': 11479433525,
 'metrics_name': 'Visit Page: Live Demo Thank you',
 'metrics_scope': 'visitor',
 'metrics_winning_direction': 'increasing',
 'start_time': '2018-09-24T18:30:44.000000Z',
 'stats_config_confidence_level': 0.9,
 'stats_config_difference_type': 'relative',
 'stats_config_epoch_enabled': False}